In [1]:
import tensorflow as tf
from tensorflow.keras.utils import image_dataset_from_directory
from tensorflow.keras.preprocessing import image
import numpy as np
from tensorflow.keras.callbacks import ReduceLROnPlateau

path = 'Rice_Leaf_AUG'

train_dataset = image_dataset_from_directory(
    path,
    validation_split=0.2,
    subset="training",
    seed=123,
    image_size=(224, 224),
    batch_size=32  
)

validation_dataset = image_dataset_from_directory(
    path,
    validation_split=0.2,
    subset="validation",
    seed=123,
    image_size=(224, 224),
    batch_size=32
)

# Store class_names before applying operations
class_names = train_dataset.class_names

AUTOTUNE = tf.data.AUTOTUNE

train_dataset = train_dataset.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
validation_dataset = validation_dataset.cache().prefetch(buffer_size=AUTOTUNE)

lr_scheduler = ReduceLROnPlateau(
    monitor='val_loss',  # Monitor the validation loss to adjust the learning rate
    factor=0.5,          # Reduce the learning rate by a factor of 0.5
    patience=3,          # Number of epochs with no improvement after which learning rate will be reduced
    min_lr=1e-6,         # Lower bound on the learning rate
    verbose=1            # Print updates about learning rate reduction
)

model = tf.keras.models.Sequential([
    tf.keras.layers.Rescaling(1./255),
    tf.keras.layers.Conv2D(32, (3,3), activation='relu', padding='same'),
    tf.keras.layers.MaxPooling2D((2,2)),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu', padding='same'),
    tf.keras.layers.MaxPooling2D((2,2)),
    tf.keras.layers.Conv2D(128, (3,3), activation='relu', padding='same', kernel_regularizer=tf.keras.regularizers.l2(0.001)),
    tf.keras.layers.MaxPooling2D((2,2)),
    tf.keras.layers.Conv2D(256, (3,3), activation='relu', padding='same'),
    tf.keras.layers.MaxPooling2D((2,2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(len(class_names), activation='softmax')
])

optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)

model.compile(
    optimizer=optimizer,
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

history = model.fit(
    train_dataset,
    validation_data=validation_dataset,
    epochs=15,
    callbacks=[lr_scheduler] 
)

model.evaluate(validation_dataset, verbose=2)

model.export('rice_final')

Found 3829 files belonging to 6 classes.
Using 3064 files for training.
Found 3829 files belonging to 6 classes.
Using 765 files for validation.
Epoch 1/15


2024-09-14 19:32:32.948683: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M3 Max
2024-09-14 19:32:32.948705: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 36.00 GB
2024-09-14 19:32:32.948716: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 13.50 GB
2024-09-14 19:32:32.948732: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-09-14 19:32:32.948742: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
2024-09-14 19:32:33.518724: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


96/96 ━━━━━━━━━━━━━━━━━━━━ 12s 91ms/step - accuracy: 0.2321 - loss: 1.8485 - val_accuracy: 0.4314 - val_loss: 1.3809 - learning_rate: 0.0010
Epoch 2/15
96/96 ━━━━━━━━━━━━━━━━━━━━ 8s 81ms/step - accuracy: 0.4686 - loss: 1.3751 - val_accuracy: 0.5190 - val_loss: 1.2445 - learning_rate: 0.0010
Epoch 3/15
96/96 ━━━━━━━━━━━━━━━━━━━━ 7s 78ms/step - accuracy: 0.5464 - loss: 1.2153 - val_accuracy: 0.5908 - val_loss: 1.0351 - learning_rate: 0.0010
Epoch 4/15
96/96 ━━━━━━━━━━━━━━━━━━━━ 7s 75ms/step - accuracy: 0.6349 - loss: 0.9901 - val_accuracy: 0.6314 - val_loss: 0.9439 - learning_rate: 0.0010
Epoch 5/15
96/96 ━━━━━━━━━━━━━━━━━━━━ 7s 75ms/step - accuracy: 0.6639 - loss: 0.8768 - val_accuracy: 0.6471 - val_loss: 0.8965 - learning_rate: 0.0010
Epoch 6/15
96/96 ━━━━━━━━━━━━━━━━━━━━ 7s 73ms/step - accuracy: 0.7155 - loss: 0.7512 - val_accuracy: 0.6706 - val_loss: 0.8986 - learning_rate: 0.0010
Epoch 7/15
96/96 ━━━━━━━━━━━━━━━━━━━━ 7s 71ms/step - accuracy: 0.7283 - loss: 0.7462 - val_accuracy: 0.7

INFO:tensorflow:Assets written to: rice_final/assets


Saved artifact at 'rice_final'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name='keras_tensor')
Output Type:
  TensorSpec(shape=(None, 6), dtype=tf.float32, name=None)
Captures:
  13656683536: TensorSpec(shape=(), dtype=tf.resource, name=None)
  13656685648: TensorSpec(shape=(), dtype=tf.resource, name=None)
  13656686032: TensorSpec(shape=(), dtype=tf.resource, name=None)
  13656686416: TensorSpec(shape=(), dtype=tf.resource, name=None)
  14345519952: TensorSpec(shape=(), dtype=tf.resource, name=None)
  14345520912: TensorSpec(shape=(), dtype=tf.resource, name=None)
  14345520336: TensorSpec(shape=(), dtype=tf.resource, name=None)
  14345521680: TensorSpec(shape=(), dtype=tf.resource, name=None)
  14345522640: TensorSpec(shape=(), dtype=tf.resource, name=None)
  14345523600: TensorSpec(shape=(), dtype=tf.resource, name=None)
  14345523408: TensorSpec(shape=(), dtype=tf.resource, name=No

In [6]:
import openvino as ov
from openvino.tools import mo
from openvino.runtime import serialize

# Path to your SavedModel directory
saved_model_path = "rice_final"

ov_model = mo.convert_model(saved_model_path, 
                            model_name="rice_model",
                            framework="tf", input_shape=[1, 224, 224, 3])  

# Specify the output directory and filenames
output_dir = "openvino_model"
xml_filename = "rice_model.xml"
bin_filename = "rice_model.bin"

# Serialize the model to IR format
serialize(ov_model, xml_filename, bin_filename)

print(f"Model converted and saved to {output_dir}/{xml_filename} and {output_dir}/{bin_filename}")

[ INFO ] MO command line tool is considered as the legacy conversion API as of OpenVINO 2023.2 release.
In 2025.0 MO command line tool and openvino.tools.mo.convert_model() will be removed. Please use OpenVINO Model Converter (OVC) or openvino.convert_model(). OVC represents a lightweight alternative of MO and provides simplified model conversion API. 
Find more information about transition from MO to OVC at https://docs.openvino.ai/2023.2/openvino_docs_OV_Converter_UG_prepare_model_convert_model_MO_OVC_transition.html
Model converted and saved to openvino_model/rice_model.xml and openvino_model/rice_model.bin
